In [1]:
# imported mysql.connector to be able to connect the database
import mysql.connector as mysql

# data management imports
import numpy as np
import pandas as pd
import seaborn as sns

In [12]:
# connect to database
# connecting to local host
mydb = mysql.connect(
    host="localhost",
    port="3306",
    user="root",
    password="ujveujveyj",
    auth_plugin="mysql_native_password",
    database='Berry_nice'
    
)

# select the correct database here
cursor = mydb.cursor()

In [13]:
# fetching all data to dataframe
cursor.execute('SELECT * FROM berries;')
table_rows = cursor.fetchall()
df = pd.DataFrame(table_rows, columns=['log_id', 'berry', 'main_tree_type', 'forest_development_class', x_])

,0,1,2,3,4,5,6,7,8,9
0,10000,cloudberry,pine,old,400000,7320000,2018,5,27,21
1,10001,cloudberry,birch,clearing,470000,7150000,2018,6,28,26
2,10002,bilberry,mixed,plantation,450000,7310000,2018,7,4,27
3,10003,bilberry,pine,old,430000,6910000,2018,7,6,27
4,10004,bilberry,pine,old,410000,7040000,2018,7,7,27
...,...,...,...,...,...,...,...,...,...,...
882,10882,lingonberry,pine,plantation,390000,6730000,2022,9,17,37
883,10883,lingonberry,pine,old,420000,7140000,2022,9,18,37
884,10884,lingonberry,spruce,plantation,300000,6900000,2022,9,20,38
885,10885,lingonberry,pine,old,570000,7720000,2022,9,21,38


In [ ]:
# Adds column names
columns = [i[0] for i in ]

In [ ]:
df[]